In [1]:
import pandas as pd
import numpy as np  
import glob  
import sys
import warnings
import os
import time

In [2]:
df = pd.read_pickle('../data/pkl/merged_filtered_0620.pkl')

In [8]:
cat_lst = pd.read_excel('../data/decline_cities_kor_list.xlsx')
cat_lst = cat_lst[['name_kor', 'type']]
cat_lst


,name_kor,type
0,서울특별시,0 Recent decline
1,서울특별시 종로구,0 Continuous decline
2,서울특별시 중구,1 Recovery
3,서울특별시 용산구,1 Recovery
4,서울특별시 성동구,0 Continuous decline
...,...,...
230,경상남도 함양군,0 Continuous decline
231,경상남도 거창군,0 Continuous decline/slowdown
232,경상남도 합천군,0 Continuous decline
233,제주특별자치도 제주시,2 Continuous growth


In [9]:
cat_lst['name_kor'] = cat_lst['name_kor'].apply(lambda x : x.split())

In [10]:
lst = []
for item  in cat_lst['name_kor']:
    try:
        lst.append(item[1])
    except:
        lst.append('pass')

In [11]:
cat_lst['city'] = lst

In [12]:
import re

idx = []
for item in cat_lst['city']:
    idx.append(re.search('시$', item) != None)

In [13]:
cat_lst_use = cat_lst[idx]
cat_lst_use[['city', 'type']]

,city,type
81,동해시,0 Continuous decline
82,태백시,0 Continuous decline
83,삼척시,0 Continuous decline
84,공주시,0 Continuous decline
85,익산시,0 Continuous decline
...,...,...
220,진주시,2 Continuous growth
221,김해시,2 Continuous growth
222,양산시,2 Continuous growth
233,제주시,2 Continuous growth


In [14]:
df_group = pd.merge(left = df, right = cat_lst_use, how = 'left', on = 'city')

In [15]:
group = df_group.groupby('type')
group = [group.get_group(x) for x in group.groups]


In [16]:
key_lst = []

for df in group:
    col = df.columns
    key = df['키워드']
    keywords = ""
    features = ""
    for item in key:
        try:
            keywords += item.replace("," , " ")
        except:
            pass
    
    key_lst.append(keywords)

In [17]:
print(len(key_lst))

5


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
T_Vec = TfidfVectorizer(min_df= 5, analyzer='word', sublinear_tf= True)
DTM = T_Vec.fit_transform(key_lst)
tfidf = pd.DataFrame(DTM.toarray(), columns = T_Vec.get_feature_names())

/home/mglee/.conda/envs/svmglee/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [43]:
idx_lst = []
for type in group:
    idx_lst.append(str(type['type'].unique()))

In [44]:
idx_lst

["['0 Continuous decline']",
 "['0 Recent decline']",
 "['0 Recent decline/Peak']",
 "['1 Recovery']",
 "['2 Continuous growth']"]

In [32]:
tfidf = tfidf.T

In [46]:
tfidf.columns = idx_lst

In [79]:
tfidf.to_csv("../result/TF_IDF_by_group.csv")

In [76]:
tfidf_by_group_df = pd.DataFrame()
for i in range(5):
    temp = tfidf.iloc[:,i].sort_values(ascending=False)
    temp = temp[:100]
    temp = temp.reset_index()
    tfidf_by_group_df = pd.concat([tfidf_by_group_df, temp], axis = 1)

In [77]:
tfidf_by_group_df

,index,['0 Continuous decline'],index,['0 Recent decline'],index,['0 Recent decline/Peak'],index,['1 Recovery'],index,['2 Continuous growth']
0,지역,0.005311,지역,0.005321,지역,0.005334,지역,0.005227,지역,0.004191
1,사업,0.005175,사업,0.005159,사업,0.005206,사업,0.005119,사업,0.004096
2,지원,0.004994,창원시,0.005064,지원,0.005058,지원,0.004963,지원,0.003973
3,추진,0.004932,지원,0.005000,경기도,0.005053,추진,0.004888,도시,0.003971
4,계획,0.004891,시장,0.004976,경기,0.005052,계획,0.004849,계획,0.003945
...,...,...,...,...,...,...,...,...,...,...
95,세계,0.004488,단체,0.004538,기관,0.004565,확대,0.004446,개선,0.003636
96,활성,0.004482,구성,0.004534,서비스,0.004564,사고,0.004440,사용,0.003635
97,공무원,0.004476,도내,0.004534,코로나19,0.004564,아파트,0.004437,투자,0.003632
98,프로그램,0.004476,대책,0.004533,현장,0.004564,구성,0.004431,단체,0.003632


In [78]:
tfidf_by_group_df.to_csv("../result/TF_IDF_by_group_top100.csv")

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
C_Vec = CountVectorizer(min_df= 5, analyzer='word')
C_DTM = C_Vec.fit_transform(key_lst)
tfidf = pd.DataFrame(DTM.toarray(), columns = T_Vec.get_feature_names())

: 

: 